In [0]:
!pip install hyperas

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.datasets import mnist
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import tpe, STATUS_OK,Trials 

In [0]:
# Loading mnist dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

X_train = train_images.reshape(train_images.shape[0], -1)
X_test = test_images.reshape(test_images.shape[0], -1)

# Normalizing Datset
X_train = X_train/255
X_test =X_test/255
  
# Encoding Labels

y_train = keras.utils.to_categorical(train_labels, 10)
y_test = keras.utils.to_categorical(test_labels, 10)
 

In [0]:
def create_model():
  
  # define model
  model = Sequential()
  model.add(Dense(512, input_dim=784, activation="relu"))
  model.add(Dropout({{uniform(0,1)}}))
  model.add(Dense({{choice([512,256,1024])}}, activation={{choice(["relu","sigmoid"])}}))
  model.add(Dropout({{uniform(0,1)}}))
  model.add(Dense(10, activation="softmax"))

  # compile model
  model.compile(loss="categorical_crossentropy",optimizer={{choice(["adam", "rmsprop", "sgd"])}}, metrics=["accuracy"])
  
  # train model
  result = model.fit(X_train, y_train, epochs={{choice([10,20])}}, batch_size=128, validation_split=0.1,verbose=5)
  
  # evaluate model
#   score = model.evaluate(X_test, y_test, verbose=0)
#   accuracy = score[1]
  accuracy = np.amax(result.history['val_acc']) 
  print('Best validation acc of epoch:', accuracy)
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [100]:
from google.colab import drive
drive.mount('/gdrive')
%ls /gdrive

Mounted at /gdrive
'My Drive'/


In [101]:
best_run, best_model = optim.minimize(model=create_model, 
                                      data=data, 
                                      algo=tpe.suggest, 
                                      max_evals=5, 
                                      trials=Trials(),
                                      notebook_name=os.path.join('..','gdrive',"My Drive", "Colab Notebooks","Hyperas_MNIST"))


>>> Imports:
#coding=utf-8

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, Activation
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import tpe, STATUS_OK, Trials
except:
    pass

try:
    from google.colab import drive
except:
    pass

try:
    import numpy as np
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0,1),
        'Dense': hp.choice('Dense', [512,256,1024]),
        'activation': hp.choice('activation', ["relu","sigmoid"]),
        'Dropout_1': hp.uniform('Dropout_1', 0,1),
        'optimizer': hp.choice('optimizer', ["adam", "rmsprop", "sgd"]),
        'epochs': hp.choice('epochs', [10,20]),
    }

>>> Data
  1: 
  2: 
 

In [102]:
best_run

{'Dense': 2,
 'Dropout': 0.587606728324542,
 'Dropout_1': 0.3746350041674067,
 'activation': 0,
 'epochs': 1,
 'optimizer': 0}

In [103]:
import numpy as np
# Metric/Performance
_, accuracy = best_model.evaluate(X_test, y_test, batch_size=128, verbose=0)
print("Accuracy on Test data is : ", accuracy)

# Predictions
pred = best_model.predict_classes(X_test, batch_size=128,verbose=0)

for i in range(10):
  print("Predicted : {} --> Expected : {}".format(pred[i], np.argmax(y_test[i])))

Accuracy on Test data is :  0.9831
Predicted : 7 --> Expected : 7
Predicted : 2 --> Expected : 2
Predicted : 1 --> Expected : 1
Predicted : 0 --> Expected : 0
Predicted : 4 --> Expected : 4
Predicted : 1 --> Expected : 1
Predicted : 4 --> Expected : 4
Predicted : 9 --> Expected : 9
Predicted : 6 --> Expected : 5
Predicted : 9 --> Expected : 9
